# Indexing - Selecting subsets of data
**Indexing** or **selecting subsets of data** is one of the most confusing aspects to pandas, which is unfortunate, because it is something done so frequently. There is so much to selecting subsets of data that I have dedicated a [7-part series](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c) to it.

Reasons why selecting subsets of data is confusing:
* **Indexing** is a confusing term. **Selecting subsets of data** is more descriptive
* Each row and column may be referenced by either their **label** or **integer location**
* This dual reference capability is powerful but confusing
* The documentation uses the term **position** instead of the more descriptive **integer location**. The indexer **`.iloc`** is an abbreviation of integer location. 
* There are six indexers, **`[]`**, **`.iloc`**, **`.loc`**, **`.ix`**, **`.at`**, **`.iat`**, that each do something different
* **`.ix`** has recently been deprecated in favor of **`.loc`** and **`.iloc`**, but old Stack Overflow answers and tutorials still show it. New questions get asked each day that use this horrible indexer.

### Getting help on `[]`, `.iloc`, `loc`
If you struggle with these indexers, I encourage you to read my detailed notebook with many practice exercises. [Selecting subsets of Data in Pandas with Exercises](../Selecting%20Subsets%20of%20Data%20in%20Pandas%20part%201.ipynb)

# `.ix` deprecation
One of the largest changes to the pandas API was the deprecation of the **`.ix`** indexer in version 0.20. The **`.ix`** indexer was versatile, ambiguous, and confusing and personally, I was very excited to see its deprecation. 

**`ix`** was able to select subsets of data by both label and integer location. For instance, if we wanted to select the math and verbal SAT scores for schools with integer location of 5, 505, and 1301 we could do the following:

In [117]:
import pandas as pd
import numpy as np

In [3]:
college = pd.read_csv('../data/college.csv', index_col=0)

In [4]:
# This is deprecated. NEVER DO THIS!!
college.ix[[5, 505, 1301], ['SATMTMID', 'SATVRMID']]

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,SATMTMID,SATVRMID
INSTNM,,
The University of Alabama,565.0,555.0
Santa Clara University,665.0,635.0
University of Northern Iowa,524.0,549.0


### Alternatives to simultaneous selection by labels and integer location
In the rare event that you need to do simultaneous selection by label and integer location, you can do one of the following:

* Convert the integer locations to labels and use **`.loc`**
* Convert the labels to integer locations and use **`.iloc`**

#### Convert integer locations to labels
To convert integer locations to labels, use the index or columns to select the correct labels.

In [177]:
labels = college.index[[5, 505, 1301]]
labels

Index(['The University of Alabama', 'Santa Clara University',
       'University of Northern Iowa'],
      dtype='object', name='INSTNM')

In [178]:
college.loc[labels, ['SATMTMID', 'SATVRMID']]

,SATMTMID,SATVRMID
INSTNM,,
The University of Alabama,565.0,555.0
Santa Clara University,665.0,635.0
University of Northern Iowa,524.0,549.0


#### Convert labels to integer locations
Converting labels to integer locations is a bit trickier. The documentation suggests the Index method **`get_loc`** for a single value and **`get_indexer`** for multiple values. These only work with Indexes containing only unique values.

In [5]:
cols = ['SATMTMID', 'SATVRMID']
ilocs = college.columns.get_indexer(cols)
ilocs

array([7, 6])

In [6]:
college.iloc[[5, 505, 1301], ilocs]

,SATMTMID,SATVRMID
INSTNM,,
The University of Alabama,565.0,555.0
Santa Clara University,665.0,635.0
University of Northern Iowa,524.0,549.0


If you were getting a single column:

In [8]:
college.iloc[[5, 505, 1301], college.columns.get_loc('SATMTMID')]

INSTNM
The University of Alabama      565.0
Santa Clara University         665.0
University of Northern Iowa    524.0
Name: SATMTMID, dtype: float64

# Assigning new values to subsets of data
The **`SettingWithCopy`** warning is a fairly common, yet important nuisance that will pop up for nearly everyone from time to time. It gets triggered whenever you select a subset of data and then in a separate operation, assign new values to that subset. This is referred to as [chained indexing in the documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html#why-does-assignment-fail-when-using-chained-indexing).

The warning arises because pandas is unable to determine if you want the original DataFrame modified or just the subset. Furthermore, sometimes the original DataFrame is modified and sometimes it isn't. It's very confusing to know what is going on. Let's see some simple, straightforward examples that activate the **`SettingWithCopy`** warning with different results of the original data.

Make a fresh copy of the college dataset so we don't modify it.

In [319]:
c1 = college.copy()
c2 = college.copy()

Select a column as a Series and as a DataFrame

In [320]:
city1 = c1['CITY']   
city2 = c2[['CITY']]

In [322]:
city1.head()

INSTNM
Alabama A & M University                   Normal
University of Alabama at Birmingham    Birmingham
Amridge University                     Montgomery
University of Alabama in Huntsville    Huntsville
Alabama State University               Montgomery
Name: CITY, dtype: object

In [323]:
city2.head()

,CITY
INSTNM,
Alabama A & M University,Normal
University of Alabama at Birmingham,Birmingham
Amridge University,Montgomery
University of Alabama in Huntsville,Huntsville
Alabama State University,Montgomery


Make the assignment that triggers the warning

In [239]:
city1.iloc[2:5] = 'NEW CITY'

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [240]:
city2.iloc[2:5] = 'NEW CITY'

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Confirm that our subset has new values

In [241]:
city1.head()

INSTNM
Alabama A & M University                   Normal
University of Alabama at Birmingham    Birmingham
Amridge University                       NEW CITY
University of Alabama in Huntsville      NEW CITY
Alabama State University                 NEW CITY
Name: CITY, dtype: object

In [242]:
city2.head()

,CITY
INSTNM,
Alabama A & M University,Normal
University of Alabama at Birmingham,Birmingham
Amridge University,NEW CITY
University of Alabama in Huntsville,NEW CITY
Alabama State University,NEW CITY


Look at original data to see what has changed

In [243]:
c1['CITY'].head()

INSTNM
Alabama A & M University                   Normal
University of Alabama at Birmingham    Birmingham
Amridge University                       NEW CITY
University of Alabama in Huntsville      NEW CITY
Alabama State University                 NEW CITY
Name: CITY, dtype: object

In [244]:
c2['CITY'].head()

INSTNM
Alabama A & M University                   Normal
University of Alabama at Birmingham    Birmingham
Amridge University                     Montgomery
University of Alabama in Huntsville    Huntsville
Alabama State University               Montgomery
Name: CITY, dtype: object

#### Conclusion
Selecting a Series creates a **view** of the original, which is then modified. Selecting a DataFrame creates a **copy** of the original.

A view is a just a pointer to the original data. A copy is a fresh new object not connected to the original.

### Double check that we get the same results with `.loc`

In [245]:
idx = ['University of Alabama at Birmingham', 'Amridge University']

In [246]:
city1.loc[idx] = 'NEW CITY 99999'

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [247]:
city2.loc[idx] = 'NEW CITY 99999'

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [248]:
c1['CITY'].head()

INSTNM
Alabama A & M University                       Normal
University of Alabama at Birmingham    NEW CITY 99999
Amridge University                     NEW CITY 99999
University of Alabama in Huntsville          NEW CITY
Alabama State University                     NEW CITY
Name: CITY, dtype: object

In [249]:
c2[['CITY']].head()

,CITY
INSTNM,
Alabama A & M University,Normal
University of Alabama at Birmingham,Birmingham
Amridge University,Montgomery
University of Alabama in Huntsville,Huntsville
Alabama State University,Montgomery


#### Conclusion
The indexer used on the view/copy is irrelevant. The only thing that matters is whether the very first selection creates a copy or a view.

## Example select rows first with `.iloc`
Let's select some rows with **`.iloc`** and then set some values on this subset.

In [273]:
c1 = college.copy()
df1 = c1.iloc[:5]  # df1 is our new subset
df1

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


Make a couple different assignments

In [276]:
df1.iloc[:2] = 99
df1['STABBR'] = 'TX'

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [277]:
c1.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,99,TX,99.0,99.0,99.0,99,99.0,99.0,99.0,99.0,...,99.0000,99.0000,99.0000,99.0000,99,99.0000,99.0000,99.000,99,99
University of Alabama at Birmingham,99,TX,99.0,99.0,99.0,99,99.0,99.0,99.0,99.0,...,99.0000,99.0000,99.0000,99.0000,99,99.0000,99.0000,99.000,99,99
Amridge University,Montgomery,TX,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.854,40100,23370
University of Alabama in Huntsville,Huntsville,TX,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.264,45500,24097
Alabama State University,Montgomery,TX,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.127,26600,33118.5


#### Conclusion
It appears that **`.iloc`** creates a view and our original data is modified

### Exercise 1
<span style="color:green; font-size:16px">Use **`.iloc`** to select the first 5 rows and columns. Is a view or copy created?</span>

In [268]:
# your code here

### Exercise 2
<span style="color:green; font-size:16px">Select a few rows and all the columns with **`.loc`**. Is a view or copy created?</span>

In [ ]:
# your code here

# Two common scenarios when assigning subsets of data
1. You want to make an assignment to a particular subset of your DataFrame but want to keep doing analysis on the entire DataFrame
2. You want to select a subset of data and store it as its own variable and modify that subset without modifying your original data.

We will cover how to properly handle each scenario with getting the **`SettingWithCopy`** warning.

## Scenario 1
For scenario 1, you won't be creating a new variable to reference the subset of data. Instead, you will use a single set of brackets to make the assignment. Let's look at a smaller sample dataset to make this clear.

In [281]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


Let's say, we want to change the color for Aaron and Dean to **`PURPLE`**. Doing this idiomatically with a single set of brackets would look like this:

In [282]:
df.loc[['Aaron', 'Dean'], 'color'] = 'PURPLE'
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,PURPLE,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,PURPLE,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


#### Incorrect versions
It's good to look at improper ways to do this so you can spot this in other code. 

In [287]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)
df['color'][['Aaron', 'Dean']] = 'PURPLE'
df

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,PURPLE,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,PURPLE,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


This was an example of **chained indexing**, which you should never do. The first indexing happens with **`df['color']`**. A temporary object is created which then indexes again with **`[['Aaron', 'Dean']]`**.

#### Another incorrect version

In [290]:
# NO WARNING! Assignment didn't complete. Looks like a bug
df = pd.read_csv('../data/sample_data.csv', index_col=0)
df.loc[['Aaron', 'Dean']]['color'] = 'PURPLE'
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


### Using boolean indexing
Change the **`score`** of all the people under 30 to 100.

In [293]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)

df.loc[df['age'] < 30, 'score'] = 100
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,100.0
Aaron,FL,red,Mango,12,120,100.0
Penelope,AL,white,Apple,4,80,100.0
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


### Exercise 1
<span style="color:green; font-size:16px">Turn to your neighbors. Explain to them why the following did not work. 
Write another incorrect version of it as well as a correct version.</span>

In [296]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)

df[df.age < 30]['score'] = 100
df

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


### Exercise 2
<span style="color:green; font-size:16px">Make the following idiomatic</span>

In [297]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)

df.loc[df['color'].isin(['red', 'green']), 'score'] = 99

In [ ]:
# your code here

### Exercise 3
<span style="color:green; font-size:16px">Make the following idiomatic</span>

In [298]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)

df[df['state'] == 'TX'][df['age'] > 30][['height', 'score']] = 99

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [299]:
# your code here

### Exercise 4
<span style="color:green; font-size:16px">Make the following idiomatic</span>

In [ ]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)

df.iloc[[6,4,2,3]].iloc[:, :3] = 'CHANGED'

### Summary of Scenario 1:
* Use exactly one set of brackets to make the assignment 
* You know you've made a mistake when you see back to back brackets like this **`][`**
* Separate row and column selection by a comma with the same set of brackets

## Scenario 2
Scenario 2 exists when you take a subset of data and want to keep working with just that subset. You may not care at all about the original DataFrame, but you probably won't want to change its data. 

In this scenario, you will use the **`copy`** method to create a fresh independent copy of your subset and then make changes to that.

In [300]:
df = pd.read_csv('../data/sample_data.csv', index_col=0)
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


Let's say we are only interested in the **`food`** and **`score`** columns and store those columns as a new variable

In [306]:
food_score = df[['food', 'score']]

Now, if we wanted to change the scores of the rows with food equal to steak or lamb to 99 we would do the following, which triggers the warning.

In [307]:
criteria = food_score['food'].isin(['Steak', 'Lamb'])
food_score.loc[criteria, 'score'] = 99
food_score

/Users/Ted/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,food,score
Jane,Steak,99.0
Niko,Lamb,99.0
Aaron,Mango,9.0
Penelope,Apple,3.3
Dean,Cheese,1.8
Christina,Melon,9.5
Cornelia,Beans,2.2


This warning was triggered by the very first step

```
>>> food_score = df[['food', 'score']]
```

The warning will be triggered for any change to this dataset. For instance, let's add a new column:

In [309]:
food_score['newcol'] = -1
food_score

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,food,score,newcol
Jane,Steak,99.0,-1
Niko,Lamb,99.0,-1
Aaron,Mango,9.0,-1
Penelope,Apple,3.3,-1
Dean,Cheese,1.8,-1
Christina,Melon,9.5,-1
Cornelia,Beans,2.2,-1


Let's look at our original DataFrame to see if it has changed

In [311]:
df

,state,color,food,age,height,score
Jane,NY,blue,Steak,30,165,4.6
Niko,TX,green,Lamb,2,70,8.3
Aaron,FL,red,Mango,12,120,9.0
Penelope,AL,white,Apple,4,80,3.3
Dean,AK,gray,Cheese,32,180,1.8
Christina,TX,black,Melon,33,172,9.5
Cornelia,TX,red,Beans,69,150,2.2


It is still in tact, but pandas does not know whether you wanted it that way. To divorce this new dataset from its original, use the **`copy`** method

In [312]:
food_score = df[['food', 'score']].copy()

criteria = food_score['food'].isin(['Steak', 'Lamb'])
food_score.loc[criteria, 'score'] = 99
food_score

,food,score
Jane,Steak,99.0
Niko,Lamb,99.0
Aaron,Mango,9.0
Penelope,Apple,3.3
Dean,Cheese,1.8
Christina,Melon,9.5
Cornelia,Beans,2.2


In [313]:
food_score['newcol'] = -1
food_score

,food,score,newcol
Jane,Steak,99.0,-1
Niko,Lamb,99.0,-1
Aaron,Mango,9.0,-1
Penelope,Apple,3.3,-1
Dean,Cheese,1.8,-1
Christina,Melon,9.5,-1
Cornelia,Beans,2.2,-1


Pandas now knows that this is independent from any other DataFrame, so no warning appears.

### Exercise 5
<span style="color:green; font-size:16px">Select the first three rows and first three columns into a new variable and then change all the values to **`CHANGED`** without getting a **`SettingWithCopy`** warning.</span>

In [314]:
# run this first
df = pd.read_csv('../data/sample_data.csv', index_col=0)

In [ ]:
# your code here

In [316]:
df._take??

In [317]:
df.iloc.__getitem__??

In [318]:
df.filter??

## No need to memorize
I don't know the rules for when pandas creates a view or a copy. I am always in either scenario 1 or 2 so either change my original DataFrame or force a copy of the subset and continue.

# Summary
* Make sure you know how to use **`[]`**, **`.iloc`**, **`.loc`**
* Never use the deprecated **`ix`**
* If you need to simultaneously select by label and integer location (rare), convert integers to labels (**`df.index[integers]`**), or labels to integers with (**`Index.get_loc/get_indexer`**) and use **`.loc`** or **`.iloc`**
* The **`SettingWithCopy`** is triggered whenever you select a subset of data and try to set a value on this subset.
* Use either an assignment with a single set of brackets or the **`copy`** method, depending on what scenario you are in to avoid the warning
* See this [excellent blog post](https://www.dataquest.io/blog/settingwithcopywarning/) from Benjamin Pryke on everything you ever wanted to know about the **`SettingWithCopy`** warning